In [2]:
import os
import torch
import json
import numpy as np
import pandas as pd
from PIL import Image
from open_clip import create_model_from_pretrained, get_tokenizer

# Load the model and tokenizer
model, preprocess = create_model_from_pretrained('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

# Set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()
print(f"Model loaded on device: {device}")

/Users/huyennguyen/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Model loaded on device: cpu


In [3]:
# Define input directories
image_folder = "../data/unified/imgs"
text_folder = "../data/unified/alt"
spec_folder = "../data/unified/specs"
output_folder = "./embeddings"
os.makedirs(output_folder, exist_ok=True)

batch_size = 50  # Adjust batch size as needed
context_length = 256

def generate_text_embeddings(folder, output_name):
    text_files = [f for f in os.listdir(folder) if f.lower().endswith(".txt") or f.lower().endswith(".json")]
    text_embeddings = []
    file_names = []
    
    for i in range(0, len(text_files), batch_size):
        batch_files = text_files[i:i + batch_size]
        texts = []
        
        for text_file in batch_files:
            text_path = os.path.join(folder, text_file)
            
            with open(text_path, "r", encoding="utf-8") as f:
                if text_file.lower().endswith(".json"):
                    try:
                        data = json.load(f)
                        text_desc = json.dumps(data)  # Convert JSON object to a string
                    except json.JSONDecodeError:
                        print(f"Warning: Could not parse JSON in {text_file}, skipping.")
                        continue
                else:
                    text_desc = f.read().strip()
            
            texts.append(text_desc)
            file_names.append(text_file)
        
        text_input = tokenizer(texts, context_length=context_length).to(device)
        
        with torch.no_grad():
            text_features = model.encode_text(text_input)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        
        text_embeddings.append(text_features.cpu().numpy())
        print(f"Processed {i + batch_size} of {len(text_files)} text files.")
    
    save_embeddings(text_embeddings, file_names, output_name)

# Function to generate image embeddings in batches
def generate_image_embeddings(folder, output_name):
    image_files = [f for f in os.listdir(folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))]
    image_embeddings = []
    file_names = []
    
    for i in range(0, len(image_files), batch_size):
        batch_files = image_files[i:i + batch_size]
        images = []
        
        for image_file in batch_files:
            image_path = os.path.join(folder, image_file)
            image = preprocess(Image.open(image_path).convert("RGB"))
            images.append(image)
            file_names.append(image_file)
        
        images_tensor = torch.stack(images).to(device)
        
        with torch.no_grad():
            image_features = model.encode_image(images_tensor)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        
        image_embeddings.append(image_features.cpu().numpy())
        print(f"Processed {i + batch_size} of {len(image_files)} image files.")
    
    save_embeddings(image_embeddings, file_names, output_name)

# Function to save embeddings as TSV and Parquet
def save_embeddings(embeddings, file_names, output_name):
    if not embeddings:
        print(f"No embeddings generated for {output_name}")
        return
    
    embeddings_np = np.vstack(embeddings)
    df = pd.DataFrame(embeddings_np, columns=[f"dim_{i}" for i in range(embeddings_np.shape[1])])
    df.insert(0, "file_name", file_names)
    
    tsv_path = os.path.join(output_folder, f"{output_name}.tsv")
    parquet_path = os.path.join(output_folder, f"{output_name}.parquet")
    
    df.to_csv(tsv_path, sep="\t", index=False)
    df.to_parquet(parquet_path, index=False)
    
    print(f"Embeddings saved: {tsv_path} and {parquet_path}")

In [4]:
# Generate embeddings
print("Generating text embeddings...")
generate_text_embeddings(text_folder, "text_embeddings")

print("Generating specification embeddings...")
generate_text_embeddings(spec_folder, "spec_embeddings")

print("Generating image embeddings...")
generate_image_embeddings(image_folder, "image_embeddings")

print("All embeddings generated and saved successfully!")

Generating text embeddings...
Processed 50 of 3200 text files.
Processed 100 of 3200 text files.
Processed 150 of 3200 text files.
Processed 200 of 3200 text files.
Processed 250 of 3200 text files.
Processed 300 of 3200 text files.
Processed 350 of 3200 text files.
Processed 400 of 3200 text files.
Processed 450 of 3200 text files.
Processed 500 of 3200 text files.
Processed 550 of 3200 text files.
Processed 600 of 3200 text files.
Processed 650 of 3200 text files.
Processed 700 of 3200 text files.
Processed 750 of 3200 text files.
Processed 800 of 3200 text files.
Processed 850 of 3200 text files.
Processed 900 of 3200 text files.
Processed 950 of 3200 text files.
Processed 1000 of 3200 text files.
Processed 1050 of 3200 text files.
Processed 1100 of 3200 text files.
Processed 1150 of 3200 text files.
Processed 1200 of 3200 text files.
Processed 1250 of 3200 text files.
Processed 1300 of 3200 text files.
Processed 1350 of 3200 text files.
Processed 1400 of 3200 text files.
Processed 

In [11]:
import pandas as pd

# Load the TSV file
tsv_path = "./embeddings/image_embeddings.tsv"
df_tsv = pd.read_csv(tsv_path, sep="\t")

# Display the first few rows
print(df_tsv.head())

                                          file_name     dim_0     dim_1  \
0  PBCA-DE-4d121008-0987-4142-a2b8-7dac58f3110a.png -0.077411 -0.007856   
1              gene_annotation_p_0_sw_1_2_s_0_7.png -0.063462  0.001907   
2  PBCA-DE-90ab64be-2229-4a8e-b864-4260b6cb39e9.png -0.065462  0.001480   
3             stacked-bar_p_0_sw_0_7_s_1_0_cc_0.png -0.076292  0.001545   
4             circular-heat_p_0_sw_1_0_s_1_2_oc.png -0.017972 -0.027335   

      dim_2     dim_3     dim_4     dim_5     dim_6     dim_7     dim_8  ...  \
0 -0.216870 -0.045800 -0.004959 -0.018328 -0.045036 -0.024274  0.015913  ...   
1 -0.058984 -0.015695 -0.003724 -0.041145 -0.021540 -0.026976  0.033540  ...   
2 -0.217632 -0.032092  0.014521 -0.028274 -0.012809 -0.046373  0.010703  ...   
3 -0.080196 -0.101816  0.004824 -0.036054  0.018174 -0.034524  0.056144  ...   
4 -0.059445 -0.013045  0.054187 -0.017061 -0.036918 -0.033271 -0.033542  ...   

    dim_502   dim_503   dim_504   dim_505   dim_506   dim_507   dim_

In [15]:
# Load the Parquet file
parquet_path = "./embeddings/spec_frequency.parquet"
df_parquet = pd.read_parquet(parquet_path)

# Display the first few rows
print(df_parquet.head())

                     file_name  aggregate -> "max"  aggregate -> "min"  \
0  AREA_sw_0_7_s_0_7_cc_0.json                   0                   0   
1  AREA_sw_0_7_s_0_7_cc_1.json                   0                   0   
2  AREA_sw_0_7_s_0_7_cc_2.json                   0                   0   
3  AREA_sw_0_7_s_0_7_cc_3.json                   0                   0   
4    AREA_sw_0_7_s_0_7_oc.json                   0                   0   

   align -> "left"  align -> "right"  alignment -> "overlay"  \
0                0                 0                       0   
1                0                 0                       0   
2                0                 0                       0   
3                0                 0                       0   
4                0                 0                       0   

   alignment -> "stack"  arrange -> "behind"  arrangement -> "horizontal"  \
0                     0                    0                            0   
1               

In [14]:
# Convert TSV to parquet

import pandas as pd

# Define file paths
tsv_path = "./embeddings/spec_frequency.tsv"
parquet_path = "./embeddings/spec_onehot.parquet"

# Read the TSV file
df = pd.read_csv(tsv_path, sep="\t")

# Save as Parquet file
df.to_parquet(parquet_path, index=False)

print(f"Converted {tsv_path} to {parquet_path}")

Converted ./embeddings/spec_frequency.tsv to ./embeddings/spec_frequency.parquet
